In [1]:
import os, json
import pandas as pd
import timeit
import sqlalchemy as alch
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from collections import Counter
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
plt.rcParams.update({'font.size': 22})

In [2]:
#SETUP PARAMETERS
topicId = '19'
topicName = 'Care Assistance and Nursing'

In [3]:
# CONNECT TO DB
engine = alch.create_engine('sqlite:///jobs.db', echo=False)

In [4]:
%%time
with engine.connect() as connection:
    result = connection.execute("select count(*) from \"main\".\"jobs.db\" ")
    number_of_rows = result.first()[0]
print(number_of_rows)

3299995
Wall time: 1.24 s


In [5]:
engine2 = alch.create_engine('sqlite:///jobs_LDA_20.db', echo=False)

In [6]:
%%time
with engine2.connect() as connection:
    result = connection.execute("select count(*) from jobs WHERE jobTopic = "+topicId)
    number_of_rows = result.first()[0]
print(number_of_rows)

212829
Wall time: 345 ms


In [7]:
%%time
ads = [None]*number_of_rows
i=0
with engine2.connect() as connection:
    result = connection.execute("select jobId from jobs WHERE jobTopic = "+topicId)
    for row in result:
        ads[i] = row[0]
        i+=1
print(ads[:10])

[37000016, 37000026, 37000038, 37000074, 37000080, 37000115, 37000128, 37000168, 37000223, 37000229]
Wall time: 699 ms


In [8]:
dates_posted = np.array([None]*number_of_rows)
expiry_dates = np.array([None]*number_of_rows)
yearly_salaries = np.array([None]*number_of_rows)

In [ ]:
%%time
# PROCESS ROWS TO GET DATES
with engine.connect() as connection:
    result = connection.execute("select jobId, datePosted, yearlyMaximumSalary, currency from \"main\".\"jobs.db\"")
    i = 0
    j = 0
    nones = 0
    for row in result:

        if (row[0] == str(ads[j])):
            try:
                dates_posted[j] = datetime.strptime(row[1],"%d/%m/%Y")
            except:
                dates_posted[j] = None
                nones+=1
                
            yearly_salaries[j] = row[2] if row[3] == 'GBP' else None
            j+=1    
            if j==number_of_rows:
                break
        i+=1  
        if i % 100000 == 0:
            print(i,"rows processed")

100000 rows processed
200000 rows processed
300000 rows processed
400000 rows processed
500000 rows processed
600000 rows processed
700000 rows processed
800000 rows processed
900000 rows processed
1000000 rows processed
1100000 rows processed
1200000 rows processed
1300000 rows processed
1400000 rows processed
1500000 rows processed
1600000 rows processed
1700000 rows processed
1800000 rows processed
1900000 rows processed
2000000 rows processed
2100000 rows processed
2200000 rows processed
2300000 rows processed
2400000 rows processed
2500000 rows processed
2600000 rows processed
2700000 rows processed
2800000 rows processed
2900000 rows processed


In [ ]:
dates_posted_filtered = [None]*(number_of_rows-nones)

In [ ]:
c=0
for i in range(len(dates_posted)):
    if dates_posted[i] != None:
        dates_posted_filtered[c] = dates_posted[i]
        c+=1

In [ ]:
print(dates_posted[:10])

In [ ]:
%%time
days = Counter(dates_posted).keys() 
posts = Counter(dates_posted).values()

In [ ]:
# PLOT
fig = plt.figure(figsize=(20,15))
ax = plt.gca()
fig.autofmt_xdate()

#formatter = mdates.DateFormatter("%Y-%m-%d")
ax.xaxis.set_major_formatter(formatter)
locator = mdates.MonthLocator()
ax.xaxis.set_major_locator(locator)
plt.title("Job adverts posted to Reed per day in "+topicName)
plt.xlabel("Date")
plt.ylabel("Number of jobs posted")
plt.plot(list(days), list(posts))

In [ ]:
# We will now redo the calculations with a weekly total instead of a daily value
print(dates_posted[78])

In [ ]:
print(dates_posted[0].weekday())

In [ ]:
# array to hold the weekly total
weekly_totals = []
# array to hold the daily counts
daily_totals = [0,0,0,0,0,0,0]
# array to hold the datetime objects for each end of week
weeks = []
# find out what weekday the first day is
w=dates_posted[0].weekday()

# initialise the running total
running_total = 0

# for every day, add the daily posts to the running total, if sunday add to the weekly total and then reset the day counter
c = 0
for i in posts: 
    running_total = running_total + i
    daily_totals[w] = daily_totals[w] + i
    c+=1
    if w==6:
        w=0
        weekly_totals.append(running_total)
        weeks.append(list(days)[c])
        running_total = 0
    else:
        w+=1
    

In [ ]:
print(len(weekly_totals))
print(daily_totals)
print(len(weeks))
print(list(days)[0])

In [ ]:
weeks = weeks[1:]
weekly_totals = weekly_totals[1:]

series = pd.Series(weekly_totals)
rolling_window = series.rolling(4)
rolling_average = rolling_window.mean()
weekly_rolling = rolling_average.tolist()
#weekly_rolling[0]= weekly_rolling[3]
#weekly_rolling[1]= weekly_rolling[3]
#weekly_rolling[2]= weekly_rolling[3]

In [ ]:
fig = plt.figure(figsize=(20,15))
ax = plt.gca()
fig.autofmt_xdate()

#formatter = mdates.DateFormatter("%Y-%m-%d")
ax.xaxis.set_major_formatter(formatter)
locator = mdates.MonthLocator()
ax.xaxis.set_major_locator(locator)
plt.title("Job adverts posted to Reed per week in "+topicName)
plt.xlabel("Date")
plt.ylabel("Number of jobs posted")
plt.plot(weeks, weekly_totals, label='Weekly total')
plt.plot(weeks, weekly_rolling, label='Rolling average over 4 weeks')
plt.legend(fontsize='xx-large',loc=3)

In [ ]:
fig = plt.figure(figsize=(10,7.5))
x = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, daily_totals)
plt.xlabel("Day of the Week")
plt.ylabel("Number of ads posted")
plt.title("Total ads posted for each day of the week in "+topicName)

plt.xticks(x_pos, x)

plt.show()

In [ ]:
%%time
nonGBP=0
total=0
lowest =  float('inf') 
highest =  float('-inf') 


for i in range(number_of_rows):
    if yearly_salaries[i] == None or yearly_salaries[i] < 1000 or yearly_salaries[i] > 5000000:
        nonGBP +=1
    else:
        total+=yearly_salaries[i]
        if yearly_salaries[i] > highest:
            highest = yearly_salaries[i]
        if yearly_salaries[i] < lowest:
            lowest = yearly_salaries[i]  
            
            
average_salary = total/(number_of_rows - nonGBP)
print("Average Salary: " +str(average_salary))
print("Highest Salary: " +str(highest))
print("Lowest Salary: " +str(lowest))

In [ ]:
data = np.array([None]*(number_of_rows-nonGBP))
j = 0
for i in range(number_of_rows):
    if yearly_salaries[i] == None or yearly_salaries[i] < 1000 or yearly_salaries[i] > 5000000:
        pass
    else:
        data[j] = yearly_salaries[i]
        j+=1
            

In [ ]:
print(data)

In [ ]:
def is_outlier(points, thresh=3.5):
    points = np.array(points)
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff.astype(float))
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [ ]:
filtered = data[~is_outlier(data)]

In [ ]:
fig = plt.figure(figsize=(10,7.5))
plt.hist(filtered, bins=20)
plt.xlabel("Yearly salary in £")
plt.ylabel("Number of ads posted")
plt.title("Histogram of yearly salaries in "+topicName)
plt.axvline(average_salary, color='r', linestyle='dashed', linewidth=1)
plt.show()